In [ ]:
!pip uninstall -y --quiet torch
!pip install vllm
!git clone https://github.com/Green0-0/light_prompter

In [ ]:
# Import required modules
from light_prompter.model import VLLMModel
from light_prompter import responder
from light_prompter.responders.prompt_basic import Prompt_Basic
from light_prompter.responders.pickcommon_custom import PickCommon_Custom
from light_prompter.sane_defaults import *

In [ ]:
from vllm import LLM, SamplingParams

# Initialize model
llm = LLM(model="Qwen/Qwen2.5-Math-7B-Instruct",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.95,
          swap_space=4,
          max_model_len=4096,
          tensor_parallel_size=2)

sampling_params = SamplingParams(temperature=0.75,
                                 min_p=0.1,
                                 max_tokens=1500,
                                 stop=["<｜end▁of▁sentence｜>", "</s>", "<|eot_id|>", "<|im_end|>", "[/INST]"])

model = VLLMModel(llm, sampling_params)

In [ ]:
# Set up PickCommon responder
def numeric_similarity(a, b):
    try:
        a_num = float(a)
        b_num = float(b)
        if a_num == b_num:
            return 0
        return -abs(a_num - b_num)  # Negative distance for sorting
    except ValueError:
        return -float('inf')  # Treat non-numeric as least similar

picker = PickCommon_Custom(
    responders={
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor),
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor), 
        Prompt_Basic(answer_extractor=regex_answer_extractor)
    },
    similarity_function=numeric_similarity
)

In [ ]:
# Define and execute query
query = generate_basic_prompt("Find the largest value of $c$ such that $1$ is in the range of $f(x)=x^2-5x+c$.", reread=True, planning=True, COT=True, answer_format="After reasoning, output your final answer in \\boxed{}.")
resp = light_prompter.responder.execute(model, picker, query)

In [ ]:
print("Verbose Output:\n" + resp.verbose_output)

In [ ]:
print("Output:\n" + resp.output)

In [ ]:
print("Answer:\n" + resp.answer)